In [15]:
%matplotlib notebook

from visualize2 import ResultParser
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.size']=8
from ipywidgets import interact,interact_manual, IntSlider, IntRangeSlider
from geometry import City
from ipywidgets import fixed,SelectionSlider
from copy import deepcopy

In [2]:
# collecting result data
rp1 = ResultParser('0831_base_fixed_ratio')
df1 = rp1.prepare_all_data(force=True)

rp2 = ResultParser('0831_base_fixed_taxis')
df2 = rp2.prepare_all_data(force=True)

Initializing ResultParser...
Done.
Initializing ResultParser...
Done.


In [10]:
def distr_gen(row):
    # request origin and destination distributions
    C = City(**row)
    
    o = []
    d = []
    for i in range(10000):
        ox,oy,dx,dy = C.create_one_request_coord()
        o.append([ox,oy])
        d.append([dx,dy])
        
    return np.histogram2d(
            np.array(o)[:,0],np.array(o)[:,1],
            range = [[0,row['n']],[0,row['m']]],
            bins = [row['n'],row['m']]
        ), np.histogram2d(
            np.array(d)[:,0],np.array(d)[:,1],
            range = [[0,row['n']],[0,row['m']]],
            bins = [row['n'],row['m']]
        )

In [11]:
df1['distr'] = df1.apply(distr_gen,axis=1)

In [12]:
df2['distr'] = df2.apply(distr_gen,axis=1)

In [13]:
def plotting_percentile(p_poor, p_rich ,d,R):
    """
    Where are the taxis from the dataset row, that have less income than the p-th percentile?
    """
    
    pr_min = p_rich[0]
    pr_max = p_rich[1]
    
    pp_min = p_poor[0]
    pp_max = p_poor[1]
    
    if R=='fixed':
        df = deepcopy(df1)
        mode = 'd'
    elif d=='fixed':
        df = deepcopy(df2)
        mode = 'R'
    
    row = df[df[mode] == eval(mode)].iloc[0]
    
    thresminr = np.percentile(row['trip_avg_price'],pr_min)
    thresmaxr = np.percentile(row['trip_avg_price'],pr_max)
    
    thresminp = np.percentile(row['trip_avg_price'],pp_min)
    thresmaxp = np.percentile(row['trip_avg_price'],pp_max)
    
    positionp = [row['position'].iloc[i] \
                for i,price in enumerate(row['trip_avg_price']) if (price > thresminp) and (price < thresmaxp)]
    
    
    positionr = [row['position'].iloc[i] \
                for i,price in enumerate(row['trip_avg_price']) if (price > thresminr) and (price < thresmaxr)]
    
    fig,ax = plt.subplots(nrows=1,ncols=4,figsize=(8,2),num = 'Taxi end positions, R=%.2f, d=%d' % (row['R'],row['d']))
    
    # poor
    ax[0].hist2d(
        [x[0] for x in positionp],[x[1] for x in positionp],
        range = [[0,row['n']],[0,row['m']]],
        bins = [row['n'],row['m']],
        cmap = 'magma'
    )
    ax[0].axis('equal')
    ax[0].set_title('Poor')
    ax[0].set_xlim(0,row['n'])
    ax[0].set_ylim(0,row['m'])
    
    # rich
    ax[1].hist2d(
        [x[0] for x in positionr],[x[1] for x in positionr],
        range = [[0,row['n']],[0,row['m']]],
        bins = [row['n'],row['m']],
        cmap = 'magma'
    )
    ax[1].axis('equal')
    ax[1].set_title('Rich')
    ax[1].set_xlim(0,row['n'])
    ax[1].set_ylim(0,row['m'])
    
    if 'request_destination_distributions' in row.index:
        if not hasattr(row['request_destination_distributions'],'dtype'):
            del row['request_destination_distributions']

    H, xedges, yedges = row['distr'][0]
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]
    ax[2].imshow(H, extent=extent, interpolation='nearest')
    ax[2].set_xlim(0,row['n'])
    ax[2].set_ylim(0,row['m'])
    ax[2].set_title('Origin')
              
    H, xedges, yedges = row['distr'][1]
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]
    ax[3].imshow(H, extent=extent, interpolation='nearest')
    ax[3].axis('equal')
    ax[3].set_xlim(0,row['n'])
    ax[3].set_ylim(0,row['m'])
    ax[3].set_title('Destination')

    plt.show()
    
#     print("INCOME VS TAXI ")
    
#     fig,axn = plt.subplots(nrows=1,ncols=4,figsize=(20,5))
#     for i,ratio in enumerate(['ratio_cruising','ratio_waiting', 'ratio_serving', 'ratio_to_request']):
#         axn[i].plot(row['trip_avg_price'],row[ratio],'bo')
#         axn[i].set_title(ratio)
#         axn[i].set_ylim(0,1)


In [16]:
interact(plotting_percentile,
         p_poor = IntRangeSlider(min=0,max=100, value=(0,10),step=10),
         p_rich = IntRangeSlider(min=0,max=100, value=(90,100),step=10),
         R = fixed('fixed'),
         d = SelectionSlider(value = 100, options=sorted(df1['d'].tolist())))

interactive(children=(IntRangeSlider(value=(0, 10), description='p_poor', step=10), IntRangeSlider(value=(90, 100), description='p_rich', step=10), SelectionSlider(description='d', options=(100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0), value=100.0), Output()), _dom_classes=('widget-interact',))

<function __main__.plotting_percentile>

In [8]:
interact_manual(plotting_percentile,
         p = IntRangeSlider(min=0,max=100, value=(90,100),step=10),
         R = SelectionSlider(value = 0.05, options=sorted(df2['R'].tolist())),
         d = fixed('fixed'))

interactive(children=(IntRangeSlider(value=(90, 100), description='p', step=10), SelectionSlider(description='R', index=1, options=(0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0), value=0.05), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.plotting_percentile>